<a href="https://colab.research.google.com/github/hpt09/ResumeParser/blob/master/Provided_Resources/EditedNameExtractionUsingSnorkel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install snorkel
!pip install nltk
!pip install spacy

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from snorkel.labeling import labeling_function
import nltk
import spacy

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df_dev = pd.read_csv(r"df_dev.csv")

In [6]:
df_dev.head(5)

,content
0,Afreen Jamadar. Active member of IIIT Committe...
1,Alok Khandai. Operational Analyst (SQL DBA) En...
2,Anvitha Rao. Automation developer. . - Email m...
3,arjun ks. Senior Program coordinator - oracle ...
4,"Arun Elumalai. QA Tester. . Chennai, Tamil Nad..."


In [7]:
from nltk import sent_tokenize as st

df_token_sentence = pd.DataFrame(columns=None)

df_token_sentence = df_dev.apply(lambda row: st(row['content']), axis=1)

In [8]:
df_token_sentence.head(2)

0    [Afreen Jamadar., Active member of IIIT Commit...
1    [Alok Khandai., Operational Analyst (SQL DBA) ...
dtype: object

In [9]:
df_token_sentence.to_csv(r"df_token_sentence.csv", header=['content'], index = False)

In [10]:
df_ts = pd.read_csv(r"df_token_sentence.csv")

In [11]:
df_ts.head(1)

,content
0,"['Afreen Jamadar.', 'Active member of IIIT Com..."


In [12]:
keep = False
df = df_ts
column = 'content'
sep = ','
indexes = list()
new_values = list()
df = df.dropna(subset=[column])
for i, presplit in enumerate(df[column].astype(str)):
    values = presplit.split(sep)
    if keep and len(values) > 1:
        indexes.append(i)
        new_values.append(presplit)
    for value in values:
        indexes.append(i)
        new_values.append(value)
new_df = df.iloc[indexes, :].copy()
new_df[column] = new_values
print(new_df)

                                               content
0                                   ['Afreen Jamadar.'
0     'Active member of IIIT Committee in Third year.'
0                                                  '.'
0                                              'Sangli
0     Maharashtra - Email me on Indeed: indeed.com/...
..                                                 ...
665                                                '.'
665                                'ACADEMIC PROJECT.'
665   'Project name:  A Case Study Undertaken to Kn...
665                                Anti-Oxidants and.'
665   'Liver Tests in the Etiology of Turner Syndro...

[86279 rows x 1 columns]


In [13]:
new_df.to_csv(r"tokenized_sentences.csv", index = False)

In [14]:
df_spacy = pd.read_csv(r"tokenized_sentences.csv")

In [15]:
df_spacy.head(3)

,content
0,['Afreen Jamadar.'
1,'Active member of IIIT Committee in Third year.'
2,'.'


In [16]:
len(df_spacy['content'])

86272

In [17]:
remove_punct = df_spacy['content'][2]

In [18]:
df_spacy = df_spacy[df_spacy.content!=remove_punct]

In [19]:
print(df_spacy.head(5))
print(len(df_spacy))

                                             content
0                                 ['Afreen Jamadar.'
1   'Active member of IIIT Committee in Third year.'
3                                            'Sangli
4   Maharashtra - Email me on Indeed: indeed.com/...
6                        'I wish to use my knowledge
69502


In [20]:
df_spacy.to_csv(r"1st_level_cleaned_sentences.csv", index=False, header=['content'])

In [21]:
df_spacy = pd.read_csv("1st_level_cleaned_sentences.csv")
df_spacy.head(10)

,content
0,['Afreen Jamadar.'
1,'Active member of IIIT Committee in Third year.'
2,'Sangli
3,Maharashtra - Email me on Indeed: indeed.com/...
4,'I wish to use my knowledge
5,skills and conceptual understanding to create...
6,'environments and work consistently achieving...
7,'and work to excellence in my work.. .'
8,'WORK EXPERIENCE.'
9,'Active member of IIIT Committee in Third year.'


In [22]:
def clean_text(text):
    text = str(text) 
    text = text.replace("'", "")
    text = text.replace("[", "")
    text = text.replace("]","")
    return text

df_spacy['content']= df_spacy['content'].apply(clean_text)

In [24]:
df_spacy.to_csv("cleaned_sentences.csv", index=False, header=['content'])

In [25]:
import pandas as pd
from snorkel.labeling import labeling_function
import nltk

import spacy
nlp = spacy.load("en_core_web_sm")

In [26]:
df_spacy = pd.read_csv(r"cleaned_sentences.csv")

In [36]:
ABSTAIN = -1
NAME = 1
SKILL = 2
EMAIL = 3
PHONE = 4

### Information Extraction

In [37]:
import re
@labeling_function()
def lf_phone_extraction(x):
    x = x.to_string()
    text = x.lower()
    pattern = r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]'    
    match = re.findall(pattern, text)
    if(len(match)> 0):
        print("Found Phone: ", match, " In: ",text)
        return PHONE
    else:
        return ABSTAIN

In [38]:
import json
@labeling_function()
def lf_skill_extraction(x):
    x = x.to_string()
    text = x.lower()
    text = text.strip()
    obj = None
    with open('cleaned_skills.json', 'r') as skills:
        data = skills.read()
        obj = json.loads(data)
    for skill in obj:
        skill = skill.lower()
        skill = (" " + skill + " ")
        if(skill in text):
            print("Found Skill: ", skill, " In: ",text)
            return SKILL
    return ABSTAIN

In [39]:
import re
@labeling_function()
def lf_email_extraction(x):
    x = x.to_string()
    ners = nlp(x)
    text = x.lower()
    pattern = '[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}'
    match = re.findall(pattern, text)
    if(len(match)> 0):
        print("Found Email: ", match, " In: ",text)
        return EMAIL
    else:
        return ABSTAIN

In [40]:
# TODO: Work on getting less false positives
from spacy.matcher import Matcher
@labeling_function()
def lf_name_extraction(x):
    x = str(x)
    nlp = spacy.load('en_core_web_sm')
    matcher = Matcher(nlp.vocab)
    nlp_text = nlp(x)
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    matcher.add('NAME', None, pattern)
    matches = matcher(nlp_text)
    if len(matches)>0:
        for match_id, start, end in matches:
            span = nlp_text[start:end]
            print("Found Name: ", span)
        return NAME
           
    else:
        return ABSTAIN
@labeling_function()
def lf_temp1(x):
    return ABSTAIN
    
@labeling_function()
def lf_temp2(x):
    return ABSTAIN

In [41]:
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel

In [45]:
lfs = [lf_phone_extraction, lf_email_extraction, lf_skill_extraction, lf_name_extraction]

In [46]:
applier = PandasLFApplier(lfs)

In [ ]:
L_train = applier.apply(df_spacy)

In [ ]:
label_model = LabelModel(cardinality=3, verbose=True)

In [ ]:
label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)

In [ ]:
df_spacy["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")

In [ ]:
df_spacy["label"]

In [ ]:
df_spacy.to_csv(path_or_buf = 'snorkel_labeled.csv',index=False)
df_spacy = df_spacy[df_spacy.label != ABSTAIN]